# Imports & Setup

In [ ]:
import datetime as dt
import time
import random
import logging
import statistics  # Für rollierende Standardabweichung

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')

# === Parameter für Dynamic Spread ===
ROLLING_WINDOW = 20            # Anzahl der Mid-Prices, über die Volatilität berechnet wird
VOL_MULTIPLIER = 1.5           # Faktor, um StdDev in Credit-Spreads zu übertragen
DEFAULT_VOL = 0.01             # Fallback-Volatilität, falls noch nicht genug Daten

# === Statische Konstanten (bleiben erhalten) ===
QUOTED_VOLUME = 10
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100

# === Connect to Exchange und Instruments holen ===
exchange = Exchange()
exchange.connect()
INSTRUMENTS = exchange.get_instruments()

# === Rolling‐Mid‐Historie initialisieren ===
mid_history = []  # Liste der letzten Mid‐Preise

# Function definitions

In [ ]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

def vwap_mid_from_top_levels(bids, asks, depth=3):
    """
    Berechnet den VWAP-basierten Mid-Preis auf Grundlage
    der obersten 'depth' Level in bids und asks.

    Parameter:
    - bids: List[OrderBookLevel], sortiert nach fallendem Preis
    - asks: List[OrderBookLevel], sortiert nach steigendem Preis
    - depth: int, Anzahl der Level pro Seite, die berücksichtigt werden

    Rückgabe:
    - float oder None (falls total_volume == 0)
    """
    total_volume = 0
    weighted_sum = 0

    # Falls das Orderbuch weniger als 'depth' Ebenen aufweist, automatisch anpassen
    actual_depth = min(depth, len(bids), len(asks))

    for level in bids[:actual_depth]:
        weighted_sum += level.price * level.volume
        total_volume += level.volume

    for level in asks[:actual_depth]:
        weighted_sum += level.price * level.volume
        total_volume += level.volume

    if total_volume == 0:
        return None

    return weighted_sum / total_volume

# Main algorithm

In [ ]:
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')
    
    for instrument in INSTRUMENTS.values():
        # Remove all existing (still) outstanding limit orders
        exchange.delete_orders(instrument.instrument_id)
    
        # Obtain order book; skip if incomplete
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue
    
        # Obtain own current position in instrument
        position = exchange.get_positions()[instrument.instrument_id]

        # Obtain best bid and ask prices from order book (für Anzeige)
        best_bid_price = instrument_order_book.bids[0].price
        best_ask_price = instrument_order_book.asks[0].price
        spread_ticks = round((best_ask_price - best_bid_price) / instrument.tick_size)

        # --- 1) Versuch Microprice Top-1, falls Spread=1 Tick und ausreichend Volumen ---
        bid_size0 = instrument_order_book.bids[0].volume
        ask_size0 = instrument_order_book.asks[0].volume

        if spread_ticks == 1 and (bid_size0 + ask_size0) > 0:
            mid_price = (best_ask_price * bid_size0 + best_bid_price * ask_size0) / (bid_size0 + ask_size0)
        else:
            mid_price = None

        # --- 2) Falls Microprice nicht verwendet wurde, prüfe Top-3 VWAP vs. einfacher Mid ---
        if mid_price is None:
            bid_vol_top3 = sum(level.volume for level in instrument_order_book.bids[:3])
            ask_vol_top3 = sum(level.volume for level in instrument_order_book.asks[:3])
            total_vol3 = bid_vol_top3 + ask_vol_top3

            if total_vol3 == 0:
                mid_price = (best_bid_price + best_ask_price) / 2.0
            else:
                rel_diff3 = abs(bid_vol_top3 - ask_vol_top3) / max(bid_vol_top3, ask_vol_top3)
                if rel_diff3 < 0.10:
                    mid_price = (best_bid_price + best_ask_price) / 2.0
                else:
                    mid_price = vwap_mid_from_top_levels(
                        instrument_order_book.bids,
                        instrument_order_book.asks,
                        depth=3
                    )
                    if mid_price is None:
                        mid_price = (best_bid_price + best_ask_price) / 2.0

        # --- 3) Volatilitäts‐basierter dynamic_credit (Spread) ---
        mid_history.append(mid_price)
        if len(mid_history) > ROLLING_WINDOW:
            mid_history.pop(0)

        if len(mid_history) >= ROLLING_WINDOW:
            rolling_std = statistics.pstdev(mid_history[-ROLLING_WINDOW:])
        else:
            rolling_std = DEFAULT_VOL

        dynamic_credit = VOL_MULTIPLIER * rolling_std

        # Calculate our fair/theoretical price based on the chosen mid_price and our current position
        theoretical_price = mid_price - PRICE_RETREAT_PER_LOT * position

        # Calculate final bid and ask prices to insert using dynamic_credit
        bid_price = round_down_to_tick(theoretical_price - dynamic_credit, instrument.tick_size)
        ask_price = round_up_to_tick(theoretical_price + dynamic_credit, instrument.tick_size)
        
        # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
        max_volume_to_buy = POSITION_LIMIT - position
        max_volume_to_sell = POSITION_LIMIT + position

        bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
        ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)

        # Wenn kein positives Volumen übrig ist, überspringen
        if bid_volume <= 0 and ask_volume <= 0:
            print(f'{instrument.instrument_id:>6s} --     NO VOLUME TO QUOTE')
            continue

        # Display information for tracking the algorithm's actions
        print(f'{instrument.instrument_id:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')
        
        # Insert new quotes
        insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume)
    
    # Wait for a few seconds to refresh the quotes
    print(f'\nWaiting for 2 seconds.')
    time.sleep(2)
    
    # Clear the displayed information after waiting (nur in Jupyter nötig)
    try:
        clear_output(wait=True)
    except NameError:
        pass